# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# Solution

## Implementation of the spelling corrector using N-gram language model

In [292]:
from collections import Counter
import string
import itertools
import random
import pandas as pd
from tqdm import tqdm

random.seed(22)

ALPHABET = string.ascii_lowercase

In [180]:
class Model:
    """
    Class for n-gram language model
    """

    def __init__(self, order: int, smoothing: float = 0.001, weights: dict[int, float] = {-1: 0.001, 1: 0.9, 2: 0.099}):
        self.order = order
        self.smoothing = smoothing
        self.weights = weights
        self.n_grams_vocab = {}
        self.vocab = set()

    def known(self, words: list[str]):
        """
        Returns the subset of `words` that are in the vocabulary
        """
        return set(word for word in words if word in self.vocab)

    def ngrams(self, text: list[str]):
        """
        Generates n-grams from `text`
        """
        return [
            tuple(text[i: i + self.order]) for i in range(len(text) - self.order + 1)
        ]

    def train_model(self, corpus: str):
        """
        Trains model on a `corpus`
        """
        split = corpus.split()
        self.vocab = set(split)
        ngram_counts = Counter(self.ngrams(split))
        total_counts = sum(ngram_counts.values())
        self.n_grams_vocab = {
            ngram: count / total_counts
            for ngram, count in ngram_counts.items()
        }

    def train_from_ngrams(self, ngram_counts: dict[tuple[str], int], total_counts: int):
        """
        Trains model from a precomputed n-gram counts
        """
        self.n_grams_vocab = {
            ngram: count / total_counts
            for ngram, count in ngram_counts.items()
        }
        self.vocab = set([word for ngram in ngram_counts.keys()
                         for word in ngram])

    def calculate_probability(self, sequence: str):
        """
        Calculates the probability of a `sequence`
        """
        probability = 1.0
        for i in range(len(sequence) - 1):
            ngram = tuple(sequence[i: i + self.order])
            if ngram in self.n_grams_vocab:
                probability *= self.n_grams_vocab[ngram]
            else:
                # Apply smoothing if ngram not found
                probability *= self.smoothing

        return probability

    def edits(self, word: str):
        """
        Generate all edits that are one edit away from `word`
        Taken from Peter Norvig's spell checker: https://norvig.com/spell-correct.html
        """
        splits = [
            (word[:i], word[i:])
            for i in range(len(word) + 1)
        ]
        deletes = [
            left + right[1:]
            for left, right in splits if right
        ]
        inserts = [
            left + c + right
            for left, right in splits
            for c in ALPHABET
        ]
        substitutions = [
            left + c + right[1:]
            for left, right in splits
            for c in ALPHABET if right
        ]
        transposes = [
            left + right[1] + right[0] + right[2:]
            for left, right in splits if len(right) > 1
        ]

        return set(deletes + inserts + substitutions + transposes)

    def edits2(self, word: str):
        """
        Generate all edits that are two edits away from `word`
        """
        return set(e2 for e1 in self.edits(word) for e2 in self.edits(e1))

    def generate_corrections(self, word: str):
        """
        Generates possible corrections for a misspelled `word`
        """
        if word in self.vocab:
            return word

        # All 1-step edits from the original word
        edits1 = self.edits(word)

        # All 2-step edits from the original word
        edits2 = self.edits2(word)

        # If no known edits, return the original word
        original = set([word])

        return {
            -1: original,
            1: self.known(edits1),
            2: self.known(edits2),
        }

    def context_correct(self, word: str, context: list[str]):
        """
        Suggests context-sensitive correction for a `word` given a `context`
        """
        best_correction, best_prob = None, 0

        probs = {}

        corrections = self.generate_corrections(word)
        if type(corrections) == str:
            return corrections, {corrections: 1}

        for candidate in corrections[-1].union(corrections[1]).union(corrections[2]):
            weights = self.weights[list(
                filter(lambda x: candidate in corrections[x], [-1, 1, 2]))[0]]
            candidate_best_prob = 0
            for context_with_word in itertools.permutations(context + [candidate]):
                context_prob = self.calculate_probability(
                    context_with_word) * weights

                if context_prob > best_prob:
                    best_correction, best_prob = candidate, context_prob
                if context_prob > candidate_best_prob:
                    candidate_best_prob = context_prob

            probs[candidate] = candidate_best_prob

        return best_correction, probs

    def correct_with_probs_print(self, word: str, context: list[str]):
        """
        Suggests context-sensitive correction for a `word` given a `context` and prints the probabilities
        """
        correction, probs = self.context_correct(word, context)
        print(f"Word: {word}")
        print(f"Context: {' '.join(context)}")
        print("Probabilities:")
        for candidate, prob in probs.items():
            print(f"  {candidate}: {prob}")
        print(f"Correction: {correction}")
        print("\n")

## Testing different n-gram models

In [181]:
corpus = "sad species dying queen king kingdom sport strength doing"

In [182]:
unigram_model = Model(order=1)
unigram_model.train_model(corpus)

unigram_model.correct_with_probs_print("dking", ["species"])
unigram_model.correct_with_probs_print("dking", ["kingdom"])
unigram_model.correct_with_probs_print("dking", ["sport"])

Word: dking
Context: species
Probabilities:
  dking: 0.0001111111111111111
  dying: 0.09999999999999999
  king: 0.09999999999999999
  doing: 0.09999999999999999
Correction: dying


Word: dking
Context: kingdom
Probabilities:
  dking: 0.0001111111111111111
  dying: 0.09999999999999999
  king: 0.09999999999999999
  doing: 0.09999999999999999
Correction: dying


Word: dking
Context: sport
Probabilities:
  dking: 0.0001111111111111111
  dying: 0.09999999999999999
  king: 0.09999999999999999
  doing: 0.09999999999999999
Correction: dying




In [183]:
bigram_model = Model(order=2)
bigram_model.train_model(corpus)

bigram_model.correct_with_probs_print("dking", ["species"])
bigram_model.correct_with_probs_print("dking", ["kingdom"])
bigram_model.correct_with_probs_print("dking", ["sport"])

Word: dking
Context: species
Probabilities:
  dking: 1e-06
  dying: 0.1125
  king: 0.0009000000000000001
  doing: 0.0009000000000000001
Correction: dying


Word: dking
Context: kingdom
Probabilities:
  dking: 1e-06
  dying: 0.0009000000000000001
  king: 0.1125
  doing: 0.0009000000000000001
Correction: king


Word: dking
Context: sport
Probabilities:
  dking: 1e-06
  dying: 0.0009000000000000001
  king: 0.0009000000000000001
  doing: 0.0009000000000000001
Correction: dying




In [184]:
trigram_model = Model(order=3)
trigram_model.train_model(corpus)

trigram_model.correct_with_probs_print("dking", ["species", "sad"])
trigram_model.correct_with_probs_print("dking", ["kingdom", "queen"])
trigram_model.correct_with_probs_print("dking", ["sport", "strength"])

Word: dking
Context: species sad
Probabilities:
  dking: 1e-09
  dying: 0.00012857142857142855
  king: 9e-07
  doing: 9e-07
Correction: dying


Word: dking
Context: kingdom queen
Probabilities:
  dking: 1e-09
  dying: 9e-07
  king: 0.00012857142857142855
  doing: 9e-07
Correction: king


Word: dking
Context: sport strength
Probabilities:
  dking: 1e-09
  dying: 9e-07
  king: 9e-07
  doing: 0.00012857142857142855
Correction: doing




## Justification of the decisions

### Which n-gram model to use?

For the spelling correction task, we need to choose the n-gram model that best captures the context of the words. The context of the words is important for the spelling correction task, because the correct spelling of a word depends on the words that come before and after it. For example, in the phrase "dking sport", the correct spelling of "dking" depends on the word "sport" that comes after it. 

With larger n-gram models, we may not have enough data to estimate the probabilities of the n-grams, and the model may become too sparse. On the other hand, with smaller n-gram models, we may not capture enough context to make accurate corrections.

I will choose the 2-gram model, because it captures enough context to make accurate corrections, and it is not too sparse.

### Which weights to assign for edit1, edit2 or absent words probabilities

For the spelling correction task, we need to assign weights to the probabilities of the edit1, edit2 and absent words. The weights should be chosen such that they reflect the likelihood of each type of error.

I decided to assign the following weights:
- word already in the vocabulary: 1 (if word already in the vocabulary - it is the most likely already correct word)
- edit1: 0.8 (one mistake is possible, so I gave it a high probability)
- edit2: 0.199 (two mistakes are less likely than one mistake, so I gave it a lower probability)
- no edits: 0.001 (almost impossible to have a word that is not in the vocabulary and has no mistakes, but I still gave it a small probability if no other options are available)

Also for checking the probability of n-grams I used smoothing with factor 0.001 to avoid zero probabilities (it is possible that some n-grams are not in the training data).

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [185]:
def read_ngrams(file: str):
    with open(file, "r") as f:
        ngram_counts = {}
        for line in f:
            split = line.split("\t")
            count = int(split[0])
            ngram = tuple(split[1:])
            ngram_counts[ngram] = count

    return ngram_counts

In [186]:
def misspell_word(word: str):
    """
    Misspells a word by randomly changing a character
    """
    mode = random.randint(2 if len(word) == 1 else 0, 4)

    if mode == 0:
        # Randomly delete a character
        idx = random.randint(0, len(word) - 1)
        return word[:idx] + word[idx + 1:]

    if mode == 1:
        # Randomly transpose two characters
        idx = random.randint(0, len(word) - 2)
        return word[:idx] + word[idx + 1] + word[idx] + word[idx + 2:]

    if mode == 2:
        # Randomly insert a character
        idx = random.randint(0, len(word))
        return word[:idx] + random.choice(ALPHABET) + word[idx:]

    if mode == 3:
        # Randomly change a character
        idx = random.randint(0, len(word) - 1)
        return word[:idx] + random.choice(ALPHABET) + word[idx + 1:]

    if mode == 4:
        # Randomly change a character to a random character
        idx = random.randint(0, len(word) - 1)
        return word[:idx] + random.choice(ALPHABET) + word[idx + 1:]

In [187]:
import re
from collections import Counter


class NorvigModel:

    def __init__(self):

        self.WORDS = Counter(self.words(open('bigrams.txt').read()))

    def context_correct(self, word: str, _: list[str]):
        """
        This function is not implemented in the Norvig model, but I added it to make the interface consistent
        """
        return self.correction(word), {}


    def words(self, text):

        return re.findall(r'\w+', text.lower())


    def P(self, word):

        "Probability of `word`."

        return self.WORDS[word] / sum(self.WORDS.values())


    def correction(self, word):

        "Most probable spelling correction for word."

        return max(self.candidates(word), key=self.P)


    def candidates(self, word):

        "Generate possible spelling corrections for word."

        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])


    def known(self, words):

        "The subset of `words` that appear in the dictionary of WORDS."

        return set(w for w in words if w in self.WORDS)


    def edits1(self, word):

        "All edits that are one edit away from `word`."

        letters = 'abcdefghijklmnopqrstuvwxyz'

        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

        deletes = [L + R[1:] for L, R in splits if R]

        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

        inserts = [L + c + R for L, R in splits for c in letters]

        return set(deletes + transposes + replaces + inserts)


    def edits2(self, word):

        "All edits that are two edits away from `word`."

        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

In [188]:
bigrams = read_ngrams("bigrams.txt")
fivegrams = read_ngrams("fivegrams.txt")

bigram_model = Model(order=2)
bigram_model.train_from_ngrams(bigrams, sum(bigrams.values()))

fivegram_model = Model(order=5)
fivegram_model.train_from_ngrams(fivegrams, sum(fivegrams.values()))

norvig_model = NorvigModel()

For the evaluation, I will use the Large Movie Review Dataset (https://ai.stanford.edu/~amaas/data/sentiment/) as a test set.

In [261]:
imdb = pd.read_csv("imdb.csv").dropna().sample(
    100, random_state=22)["review"].values.tolist()
imdb[0]

'I know John Singleton\'s a smart guy \'coz he made Boyz N The Hood, so how did he write and direct this? It\'s like the pilot of a bad "going away to college for the first time" teen soap, a parade of boring stereotypes and cliches with some gratuitous violence thrown in to make it a commercial proposition, I guess. Who would\'ve guessed the date-rape victim would dump sausage for seafood? The angry loner would be preyed upon by a group of Neo-Nazis (and would be roomed-up with a black AND a Jew - just for laughs!) Even Laurence Fishburne\'s creepy reactionary history Professor just irritated me and I love the guy, it\'s like everyone involved with this movie just lost the plot. Except Busta Rhymes, of course. Big ups.'

In [283]:
def evaluate_model(model, data):
    correct = 0
    total = 0

    for review in tqdm(data):
        words = review.split()
        # remove punctuation and lowercase
        words = [
            word.translate(str.maketrans("", "", string.punctuation)).lower()
            for word in words
        ]
        words = [word for word in words if word]

        random_word_idx = random.randint(0, len(words) - 1)
        while len(words[random_word_idx]) < 5:
            random_word_idx = random.randint(0, len(words) - 1)
        word = words[random_word_idx]
        context = (
            words[max(0, random_word_idx - 2): random_word_idx]
            + words[random_word_idx + 1: min(len(words), random_word_idx + 2)]
        )
        misspelled_word = misspell_word(word)

        correction, _ = model.context_correct(misspelled_word, context)
        if correction == word:
            correct += 1
        total += 1

    return correct / total

In [251]:
evaluate_model(bigram_model, imdb)

100%|██████████| 100/100 [00:10<00:00,  9.55it/s]


0.75

In [293]:
evaluate_model(fivegram_model, imdb)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:08<00:00, 11.49it/s]


0.72

In [302]:
evaluate_model(norvig_model, imdb)

100%|██████████| 100/100 [00:01<00:00, 54.27it/s]


0.72

My model performance is slightly better than Norvig's corrector. In my opinion, it can be even better, but dataset is not very high quality for checking the accuracy of the model, since even in the original reviews there may be errors (that is, in the context) and the model may not be able to correct them accurately.

### Example where context model is better than the Norvig's corrector

I want to correct `holme in door` to `home in door`. The Norvig's corrector will correct `holme` to `home`, which is not correct. The context model is able to correct this mistake.

In [303]:
norvig_model.correction("holme")

'home'

In [304]:
bigram_model.context_correct("holme", ["in", "door"])[0]

'hole'